# Reinforcement Learning

In [1]:
#pip install plotly

In [2]:
#pip install --upgrade nbformat

In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import matplotlib.pyplot as plt

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
prices = np.loadtxt(
    '../data/prices_btc_Jan_11_2020_to_May_22_2020.txt',
    dtype=float
    )

In [ ]:
prices[:10]

In [ ]:
len(prices)

In [ ]:
df = pd.DataFrame({"prices": prices})

df.sample(10)

In [ ]:
sns.lineplot(
    df,
)

In [ ]:
def buy(btc_price, btc, money):
    if(money != 0):
        btc = (1 / btc_price ) * money
        money = 0
    return btc, money


def sell(btc_price, btc, money):
    if(btc != 0):
        money = btc_price * btc
        btc = 0
    return btc, money


def wait(btc_price, btc, money):
    # do nothing
    return btc, money

In [ ]:
np.random.seed(1)

# set of actions that the user could do
actions = {
    'buy': buy,
    'sell': sell,
    'wait': wait
}

actions_to_nr = { 'buy' : 0, 'sell' : 1, 'wait' : 2 }
nr_to_actions = { k:v for (k,v) in enumerate(actions_to_nr) }

nr_actions = len(actions_to_nr.keys())
nr_states = len(prices)

# q-table = reference table for our agent to select the best action based on the q-value
q_table = np.random.rand(nr_states, nr_actions)

In [ ]:
actions_to_nr

In [ ]:
def get_reward(before_btc, btc, before_money, money):
    reward = 0
    if(btc != 0):
        if(before_btc < btc):
            reward = 1
    if(money != 0):
        if(before_money < money):
            reward = 1
            
    return reward

In [ ]:
def choose_action(state):
    if np.random.uniform(0, 1) < eps:
        return np.random.randint(0, 2)
    else:
        return np.argmax(q_table[state])

In [ ]:
def take_action(state, action):
    return actions[nr_to_actions[action]](prices[state], btc, money)

In [ ]:
def act(state, action, theta):
    btc, money = theta
    
    done = False
    new_state = state + 1
    
    before_btc, before_money = btc, money
    btc, money = take_action(state, action)
    theta = btc, money
    
    reward = get_reward(before_btc, btc, before_money, money)
    
    if(new_state == nr_states):
        done = True
    
    return new_state, reward, theta, done

In [ ]:
reward = 0
btc = 0
money = 100

theta = btc, money

In [ ]:
# exploratory
eps = 0.3

n_episodes = 20
min_alpha = 0.02

# learning rate for Q learning
alphas = np.linspace(1.0, min_alpha, n_episodes)

# discount factor, used to balance immediate and future reward
gamma = 1.0

In [ ]:
rewards = {}

for e in range(n_episodes):
    
    total_reward = 0
    
    state = 0
    done = False
    alpha = alphas[e]
    
    while(done != True):

        action = choose_action(state)
        next_state, reward, theta, done = act(state, action, theta)
        
        total_reward += reward
        
        if(done):
            rewards[e] = total_reward
            print(f"Episode {e + 1}: total reward -> {total_reward}")
            break
        
        q_table[state][action] = q_table[state][action] + alpha * (reward + gamma *  np.max(q_table[next_state]) - q_table[state][action])

        state = next_state

In [ ]:
plt.ylabel('Total Reward')
plt.xlabel('Episode')
plt.plot([rewards[e] for e in rewards.keys()])

In [ ]:
state = 0
acts = np.zeros(nr_states)
done = False

while(done != True):

        action = choose_action(state)
        next_state, reward, theta, done = act(state, action, theta)
        
        acts[state] = action
        
        total_reward += reward
        
        if(done):
            break
            
        state = next_state

In [ ]:
buys_idx = np.where(acts == 0)
wait_idx = np.where(acts == 2)
sell_idx = np.where(acts == 1)

In [ ]:
wait_idx

In [ ]:
buys_idx

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(buys_idx[0], prices[buys_idx], 'bo', markersize=0.5)
plt.plot(sell_idx[0], prices[sell_idx], 'ro', markersize=0.5)
plt.plot(wait_idx[0], prices[wait_idx], 'yo', markersize=0.5)

In [ ]:
b_idx = buys_idx[0] if isinstance(buys_idx, tuple) else buys_idx
s_idx = sell_idx[0] if isinstance(sell_idx, tuple) else sell_idx
w_idx = wait_idx[0] if isinstance(wait_idx, tuple) else wait_idx

# --- Configuración del Gráfico ---
plt.style.use('seaborn-v0_8-muted') 
plt.figure(figsize=(20, 10))

# 1. Trazar la línea de precio (fondo)
plt.plot(prices, color='black', alpha=0.15, linewidth=2, label='Precio')

# 2. Trazar puntos de 'Wait' (Esperar) - Muy pequeños y tenues
plt.scatter(w_idx, prices[w_idx], color='gray', s=1, alpha=0.05, label='Wait')

# 3. Trazar señales de Compra (Triángulos verdes hacia arriba)
plt.scatter(b_idx, prices[b_idx], color='green', marker='^', s=1, alpha=0.8, label='Buy')

# 4. Trazar señales de Venta (Triángulos rojos hacia abajo)
plt.scatter(s_idx, prices[s_idx], color='red', marker='v', s=10, alpha=0.8, label='Sell')

# --- Personalización y Estética ---
plt.title('Visualización de Estrategia de Trading', fontsize=16, fontweight='bold')
plt.xlabel('Índice / Tiempo', fontsize=12)
plt.ylabel('Precio', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.4)

# --- Corrección de la Leyenda (El error anterior) ---
lgnd = plt.legend(loc="upper left", scatterpoints=1, fontsize=12, frameon=True)

for handle in lgnd.legend_handles:
    # Si es un objeto de Scatter (PathCollection)
    if hasattr(handle, 'set_sizes'):
        handle.set_sizes([20.0])
    # Si es un objeto de Línea (Line2D)
    elif hasattr(handle, 'set_markersize'):
        handle.set_markersize(10.0)

plt.tight_layout()
plt.show()